<a href="https://colab.research.google.com/github/kangmg/XequiNet/blob/master/notebook/xequinet_ase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try: import xequinet
except ModuleNotFoundError:
    !pip install git+https://github.com/kangmg/XequiNet.git

  Cloning https://github.com/kangmg/XequiNet.git to /tmp/pip-req-build-l7eiat1c
  Running command git clone --filter=blob:none --quiet https://github.com/kangmg/XequiNet.git /tmp/pip-req-build-l7eiat1c
  Resolved https://github.com/kangmg/XequiNet.git to commit 633d1ad25140ddeff5d23a515bc443d9e4b525c2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.2/447.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.7 MB/s eta 0:00:00
  Created wheel fo

In [1]:
#@title Install conda

try:
  import condacolab
  condacolab.check()
except:
  %pip install condacolab
  import condacolab
  condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
# xtb calculator used for delta learning model
!conda install -q tblite
!conda install -q tblite-python

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



In [3]:
# re-check tblite is installed
try:
  from tblite.ase import TBLite
except:
  print('tblite is not callable.\n')
  !conda install -q tblite
  !conda install -q tblite-python

  print('\nRestart session . . .')

  import os
  os._exit(0)

In [5]:
# convert pt to jit

# direct learning model
!xeqjit -c spice_1109.pt -o md.jit --md
!xeqjit -c spice_1109.pt -o geo.jit

# delta learning model
!xeqjit -c d-spice_1111.pt -o d_md.jit --md
!xeqjit -c d-spice_1111.pt -o d_geo.jit

/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")
/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")
/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")
/usr/local/lib/python3.10/dist-packages/xequinet/interface/__init__.py:17: UserWarning: i-PI is not installed, i-PI driver will not be performed.
  warnings.warn("i-PI is not installed, i-PI driver will not be performed.")


In [6]:
import warnings
warnings.filterwarnings('ignore')

<br>
<br>
<font color=skyblue size=5> direct learned model </font>
<br>
<br>


In [7]:
# md type model ase test

from xequinet.interface import XeqCalculator
from ase.build import molecule

atoms = molecule('C6H6')

calc = XeqCalculator(ckpt_file='md.jit', model_type='md')

atoms.calc = calc

atoms.get_potential_energy()

-6324.118982349497

In [8]:
# geometry type model ase test

from xequinet.interface import XeqCalculator
from ase.build import molecule

atoms = molecule('C6H6')

calc = XeqCalculator(ckpt_file='geo.jit', model_type='geometry')

atoms.calc = calc

atoms.get_potential_energy()

-6324.118975673778

<br>
<br>
<font color=skyblue size=5> $\Delta$-learned model </font>
<br>
<br>


In [9]:
# geo type delta learned model
from ase.calculators.mixing import SumCalculator
from xequinet.interface import XeqCalculator
from tblite.ase import TBLite
from ase.build import molecule

atoms = molecule('C6H6')

delta_calc = SumCalculator([
    XeqCalculator(ckpt_file='d_geo.jit', model_type='geometry'), # geo type
    TBLite(verbosity=0, method='GFN2-xTB'), # xtb
])

atoms.calc = delta_calc

atoms.get_potential_energy()

-6324.108374815414

In [10]:
# md type delta learned model

from ase.calculators.mixing import SumCalculator
from xequinet.interface import XeqCalculator
from tblite.ase import TBLite
from ase.build import molecule

atoms = molecule('C6H6')

delta_calc = SumCalculator([
    XeqCalculator(ckpt_file='d_md.jit', model_type='md'), # md type
    TBLite(verbosity=0, method='GFN2-xTB'), # xtb
])

atoms.calc = delta_calc

atoms.get_potential_energy()

-6324.1083759479025